This is Everett's starter file for Project 4: Machine Learning. Our topic is breast cancer risk.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer(as_frame=True)
data.target[[10, 50, 85]]


TypeError: load_breast_cancer() got an unexpected keyword argument 'as_frame'